# Student Admission - MRSort and NCS
Ariane Dalens  
Lucas Sor  
Magali Morin  

## Générateur de dataset

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(123)


def generateur(N, m):
    """
    Crée un jeu de données de taille N sur la base de 12 notes 

    args:
    N : taille du jeu de données 
    m : nombre de notes

    output: 
    df : jeu de données 
    betas : frontière
    weights : weights du modèle
    lbd : lambda du modèle
    """

    # Générer la frontière betas
    betas = rng.integers(low=8, high=15, size=m)

    # Générer les weights et les normaliser
    weights = rng.random(m)
    weights = weights / weights.sum()

    # Générer lambda
    lbd = rng.random()

    # Générer df - sigmas: notes des students
    sigmas = rng.integers(low=0, high=21, size=(N, m))
    df = pd.DataFrame(sigmas)
    df['accepted'] = df.apply(
        lambda x: classifier(x, betas, weights, lbd), axis=1)

    return df, betas, weights, lbd


def classifier(sigma, betas, weights, lbd):
    """
    Fonction de classification pour un élève alpha 

    args:
    alpha : notes d'un élève
    betas : frontière
    weights : weights du modèle
    lbd : lambda du modèle

    output: 
    1 si l'élève est accepté
    0 sinon 
    """
    return ((sigma >= betas)*weights).sum() >= lbd

def analyse_dataset(df):
    """
    Fonction d'analyse d'un dataset

    args:
    df : jeu de données 
    """
    print('---------Analyse----------')
    print(f"Nombre d'élèves acceptés: {df.accepted.sum()}")
    print("% d'élèves acceptés {:.2f} %".format(df.accepted.sum()/len(df)))
    #plt.hist(df[0])
    pass

df, betas, weights, lbd = generateur(150, 10)
analyse_dataset(df)
df

---------Analyse----------
Nombre d'élèves acceptés: 40
% d'élèves acceptés 0.27 %


,0,1,2,3,4,5,6,7,8,9,accepted
0,9,19,15,4,17,16,4,10,16,4,True
1,5,3,0,10,0,12,8,3,7,0,False
2,3,9,1,15,5,19,3,13,10,19,False
3,12,18,9,4,3,18,4,15,11,5,False
4,8,16,9,18,19,6,11,11,11,1,True
...,...,...,...,...,...,...,...,...,...,...,...
145,4,0,11,17,0,3,18,12,16,12,False
146,19,4,17,9,0,8,8,10,3,17,False
147,6,19,20,1,15,9,10,6,8,16,False
148,12,7,14,10,15,14,18,13,15,15,True


In [ ]:
import numpy as np
from collections import Counter


class GradesGenerator():
    def __init__(self, size: int = 100, nb_classes: int = 1, nb_grades: int = 4, lbd: float = None, weights: np.ndarray = None, betas: np.ndarray = None, seed: int = None, noise: float = None):
        self.noise = noise
        self.seed = seed
        if seed is None:
            self.seed = np.random.random_integers(1,100)
        self.size = size
        self.lbd = lbd
        self.nb_classes = nb_classes
        self.nb_grades = nb_grades
        if lbd is None:
            rng = np.random.default_rng(self.seed)
            self.lbd = rng.uniform(0.2, 0.8)
        self.weights = weights
        if weights is None:
            self.weights = self.generate_weights()
        self.betas = betas
        if betas is None:
            self.betas = self.generate_betas()
        if noise is None:
            rng = np.random.default_rng(self.seed)
            self.noise = rng.uniform(0.01, 0.1)

    def generate_weights(self):
        """
        Generate weights based on a normal distribution
        Returns :
               weights (array<float>) : weights between 0 and 1 
        """
        rng = np.random.default_rng(self.seed)
        weights = abs(rng.standard_normal(self.nb_grades))
        weights /= weights.sum()
        return weights

    def generate_betas(self):
        """
        Generate betas between 8 and 15 based on a uniform distribution
        Returns :
               betas (array<int>) : frontiers of grade
        """
        rng = np.random.default_rng(self.seed)
        betas = rng.integers(low=8, high=15, size=self.nb_grades)
        return np.array(betas)

    def classifier(self, grades):
        """
        Classifies grades
        Returns :
               admissions (array<bool>) : True or False based on admission
        """
        if self.noise > 0:
            print('Adding {:.2f} % of noise'.format(self.noise*100)) 
            admissions = []
            for grade in grades: 
                tirage = np.random.binomial(1, self.noise)
                if tirage:
                    admissions += [((grade >= self.betas)*self.weights).sum() <= self.lbd]
                else:
                    admissions += [((grade >= self.betas)*self.weights).sum() >= self.lbd]
            admissions = np.array(admissions)
        else: 
            admissions = np.array([((grade >= self.betas)*self.weights).sum() >= self.lbd for grade in grades])
        return admissions

    def generate_grades(self):
        """
        Generate grades based on a uniform distribution between 0 and 20 
        Returns :
              grades (array<array<int>>) : grades of students
              admissions (array<bool>) : True or False based on admission
        """
        rng = np.random.default_rng(self.seed)
        grades = rng.integers(low=0, high=21, size=(self.size, self.nb_grades))
        admissions = self.classifier(grades)
        return grades, admissions
    
    def analyze_gen(self):
        """
        Analyze the generator
        Returns :
            None
        """
        print('---------Analyze----------')
        print(f"Lambda: {self.lbd}")
        print(f"Weights: {self.weights}")
        print(f"Betas: {self.betas}")
        grades, admissions = self.generate_grades()
        print(f"Got-in: {dict(Counter(admissions))}")
        print("% Got-in: {:.2f} %".format(admissions.sum()/self.size))
        print('--------------------------')
        pass


## Programme linéaire

In [ ]:
from gurobipy import *
import time
import numpy as np
from collections import Counter
from sklearn.metrics import f1_score


class MRSort_Solver:
    def __init__(self, generator, epsilon: float = 1e-6, M: int = 1e2):
        """
        Initialize the solver
        """
        self.gen = generator
        self.grades, self.admission = generator.generate_grades()
        self.model = Model("MR-sort")

        # Constants
        self.size = self.gen.size
        self.nb_grades = self.gen.nb_grades
        self.epsilon = epsilon
        self.M = M

        # time to solve
        self.time = None

        # ----- Gurobi variables ----
        self.obj = self.model.addVar()  # Sum(Sigma_s) objective
        # sigmas for each student (in A*)
        self.A = self.model.addMVar(shape=self.size, lb=0, ub=0.5)
        # sigmas for each student (in R*)
        self.R = self.model.addMVar(shape=self.size, lb=0,  ub=0.5)

        # weights
        self.weights = self.model.addMVar(shape=self.nb_grades, lb=0, ub=1)
        # betas
        self.betas = self.model.addMVar(shape=(self.nb_grades))
        # lambda
        self.lbd = self.model.addVar(lb=0.1, ub=1)

        # student weights
        self.weights_ = self.model.addMVar(
            shape=(self.size, self.nb_grades), lb=0, ub=1)
        # delta
        self.deltas = self.model.addMVar(
            shape=(self.size, self.nb_grades), vtype=GRB.BINARY)

    def set_constraint(self, objective):
        """
        Set the contraints of the model, please refer to the README.md for more details
        """

        # Margins in A*
        self.model.addConstrs((
            quicksum(self.weights_[j, i] for i in range(
                self.nb_grades)) - self.lbd - self.A[j] == 0
        ) for j in range(self.size) if self.admission[j] == True
        )

        # Margins in R*
        self.model.addConstrs((
            quicksum(self.weights_[j, i] for i in range(
                self.nb_grades)) - self.lbd + self.R[j] == - self.epsilon
        ) for j in range(self.size) if self.admission[j] == False
        )

        # Grades and betas-frontiers
        self.model.addConstrs((
            (self.M*(self.deltas[j, :] - np.ones(self.nb_grades)) <= self.grades[j, :] - self.betas))
            for j in range(self.size))

        self.model.addConstrs((
            self.grades[j, :] - self.betas <= self.M*self.deltas[j, :] - self.epsilon*np.ones(self.nb_grades))
            for j in range(self.size))

        # Weights constraint
        self.model.addConstrs((
            self.weights >= self.weights_[j])
            for j in range(self.size)
        )

        # Weights sum equals 1
        self.model.addConstr(
            quicksum(self.weights[k] for k in range(self.nb_grades)) == 1)

        # Delta constraints
        self.model.addConstrs((
            self.deltas[j] >= self.weights_[j])
            for j in range(self.size)
        )

        self.model.addConstrs((
            self.weights_[j] >= self.deltas[j] + self.weights - np.ones(self.nb_grades))
            for j in range(self.size)
        )

        if objective == 'MaxMin':
            # Objective is the min margin
            self.model.addConstrs((self.obj <= self.A[j]) for j in range(
                self.size) if self.admission[j] == True)
            self.model.addConstrs((self.obj <= self.R[j]) for j in range(
                self.size) if self.admission[j] == False)
        elif objective == 'Sum':
            # Objective is the sum of margins in A* and R*
            self.model.addConstr(self.obj == quicksum(self.A[j] for j in range(self.size) if self.admission[j] == True)
                                 + quicksum(self.R[j] for j in range(self.size) if self.admission[j] == False))
        else:
            print('Error objective should be MaxMin or Sum')

    def solve(self):
        """
        Solve the model
        """
        start = time.time()
        self.model.update()
        self.model.setObjective(self.obj, GRB.MAXIMIZE)
        self.model.params.outputflag = 0  # 0 means without verbose
        self.model.optimize()
        end = time.time()
        self.time = end - start

    def get_results(self):
        """
        Print results of the solver
        Returns :
            f1_score_ (float): f1-score of the solution
            accuracy_ (float): accuracy of the solution
            time (float): time spent trying to find the optimum
            error_count (int): 1/0 based on if gurobi converges or not 
        """
        try:
            results = ((self.grades > self.betas.X) *
                       self.weights.X).sum(axis=1) > self.lbd.X
            f1_score_ = f1_score(self.admission, results)
            accuracy_ = sum([results[i] == self.admission[i]
                            for i in range(len(results))])/len(results)

            print(f"results:\n")
            print(f"Objective: {self.obj.X}")
            print(f"Lambda: {self.lbd.X}")
            print(f"Weights: {self.weights.X}")
            print(f"Betas: {self.betas.X}")
            print(f"Results: {dict(Counter(results))}")
            print("Precision: {:.2f} %\n".format(accuracy_*100))
            print("F1-score:  {:.2f} %\n".format(f1_score_*100))
            error_count = 0
            return f1_score_, accuracy_, self.time, error_count

        except GurobiError:
            print("WARNING: Gurobi didn't find a solution")
            error_count = 1
            return 0, 0, 0, error_count

    def check_constraint(self):
        """
        Check if contraints are respected - debug function
        """
        # Margins in A* == 0
        print([(sum(self.weights_.X[j, i] for i in range(self.nb_grades)) - self.lbd.X -
              self.A.X[j]) == 0 for j in range(self.size) if self.admission[j] == True])
        # Margins in R* == -epsilon
        print([(sum(self.weights_.X[j, i] for i in range(self.nb_grades)) - self.lbd.X + self.R.X[j])
              == - self.epsilon for j in range(self.size) if self.admission[j] == False])

        # Grades and beta frontier >= 0
        print([-(self.M*(self.deltas.X[j, :] - np.ones(self.nb_grades)) -
              self.grades[j, :] + self.betas.X) >= 0 for j in range(self.size)])
        print([-(self.grades[j, :] - self.betas.X - self.M*self.deltas.X[j, :] +
              self.epsilon*np.ones(self.nb_grades)) >= 0 for j in range(self.size)])

        # Weights constraint >=0 (=0 ou 1)
        print([(self.weights.X - self.weights_.X[j])
              >= 0 for j in range(self.size)])

        # Delta constraints >=0
        print([(self.deltas.X[j] - self.weights_.X[j])
              >= 0 for j in range(self.size)])
        print([(self.weights_.X[j] - self.deltas.X[j] - self.weights.X +
              np.ones(self.nb_grades)) >= 0 for j in range(self.size)])


In [ ]:
# Test Generator
import sys
sys.path.append('./')

from generator import GradesGenerator

size = 100
nb_grades = 5
noise = 0

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise)
grades,admission = gen.generate_grades()
gen.analyze_gen()

# Test MR-Sort
from models import MRSort_Solver

MRSort_solv = MRSort_Solver(gen)
MRSort_solv.set_constraint('MaxMin')
MRSort_solv.solve()
f1_score_, accuracy_, time = MRSort_solv.get_results()



## Solver SAT - gophersat

Le solveur **gophersat** est un programme open-source écrit en [Go](https://golang.org/). Ses sources sont accessibles sur [https://github.com/crillab/gophersat](https://github.com/crillab/gophersat). Une version compilée est disponible sur le EDUNAO du cours. C'est un solveur SAT, MAX-SAT et pseudo-booléen. Il est également capable de faire du comptage de modèles.

In [ ]:
#Construction du DIMCS et Résolution

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], {i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 
